In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from google import genai
import anthropic
from IPython.display import Markdown, display, update_display

In [3]:
load_dotenv(override=True)
openai_key = os.getenv("OPENAI_API_KEY")
gemeni_key = os.getenv("GEMINI_API_KEY")
anthropic_key = os.getenv("ANTHROPIC_API_KEY")

In [4]:
openai = OpenAI(api_key=openai_key)
claude = anthropic.Anthropic(api_key=anthropic_key)

### Asking LLMs to tell a joke

In [5]:
system_msg = "You are an assitant that is great at telling jokes"
user_prompt = "Tell a dad joke"

In [6]:
prompts = [
    {"role":"system", "content":system_msg},
    {"role":"user", "content":user_prompt}
]

In [7]:
completions = openai.chat.completions.create(
    model = "gpt-4o",
    messages=prompts,
    temperature=0.99
)
print(completions.choices[0].message.content)

Why can't you give Elsa a balloon? 

Because she will let it go!


In [8]:
message = claude.messages.create(
    model = "claude-3-5-sonnet-latest",
    max_tokens= 200,
    temperature= 0.7,
    system=system_msg,
    messages=[
        {"role":"user", "content":user_prompt}
    ]
)
print(message.content[0].text)

Here's a classic dad joke for you:

What do you call a bear with no teeth?

A gummy bear! 😄


In [9]:
result = claude.messages.stream(
    model= "claude-3-7-sonnet-latest",
    max_tokens= 200,
    temperature= 0.99,
    system= system_msg,
    messages=[
        {"role":"user", "content": user_prompt}
    ]
)

with result as stream:
    for text in stream.text_stream:
        print(text,end="",flush=True)

Why don't scientists trust atoms?

Because they make up everything!

*Ba-dum-tss* 🥁

In [10]:
from google.genai import types
gemeni = genai.Client(
    api_key=gemeni_key
    )
response = gemeni.models.generate_content(
    contents=user_prompt,
    model= "gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction = system_msg,
        max_output_tokens=500,
        temperature=0.99
        )
    )
print(response.text)

Why don't scientists trust atoms?

Because they make up everything!



In [11]:
prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [12]:
stream = openai.chat.completions.create(
    model="gpt-4o",
    messages=prompts,
    temperature=0.99,
    stream=True
)
reply = ""
display_handle = display(Markdown(""),display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ""
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply),display_id=display_handle.display_id)

To determine if a business problem is suitable for a Large Language Model (LLM) solution, consider the following steps and criteria:

### 1. Problem Identification

- **Text-Related**: Ensure the problem fundamentally relies on processing natural language. LLMs are tailored for text generation, comprehension, summarization, translation, and more.

- **Complexity of Task**: Evaluate if the task requires understanding context, nuances, or generating human-like text. Simple string matching or rule-based automation might be addressed more efficiently by lighter models.

### 2. Data Availability

- **Text Data Volume**: Confirm there is enough quality text data. LLMs require significant amounts of data for fine-tuning if you're going beyond pre-trained capabilities.

- **Domain-Specific Data**: Consider whether there is domain-specific data available that needs understanding, like financial reports, legal documents, or medical records.

### 3. Goal Definition

- **Goal Clarity**: Define the objectives clearly. Whether it's customer service, sentiment analysis, or content generation, a clear goal will help in deciding an LLM approach.

- **Expected Outcome**: Assess if the expected outcome is feasible with an LLM. Tasks requiring reasoning beyond current LLM capabilities may not be suitable.

### 4. Technical Capabilities

- **Infrastructure**: Evaluate if you have the right technical infrastructure to deploy and maintain an LLM. They require substantial computational resources.

- **Scalability**: Determine whether the solution needs to scale. LLMs can handle large volumes of text, which is beneficial for high-scale applications.

### 5. Cost-Benefit Analysis

- **Cost Consideration**: Weigh the cost of deploying an LLM against the expected benefits. LLMs can be expensive to train and maintain.

- **Efficiency Gains**: Analyze if the LLM improves efficiency significantly over existing solutions.

### 6. Ethical and Privacy Concerns

- **Data Privacy**: Ensure that the use of text data complies with all relevant privacy regulations and ethical standards.

- **Bias and Fairness**: Consider potential biases in the language model outputs and how they might affect your business solutions.

### 7. Iterative Testing

- **Prototype and Validate**: Start with a small prototype to test viability. 

- **Iterative Feedback**: Use iterative testing with feedback loops to refine the LLM deployment, measuring against KPIs.

### Conclusion

Evaluating a business problem for an LLM solution involves a careful assessment of the task's nature, data requirements, goal alignment, technical feasibility, cost implications, and ethical considerations. By critically analyzing these factors, you can make an informed decision on utilizing an LLM effectively.

In [13]:
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very hard to get girl type; \
you flirtatiously disagree with anything in the conversation and what others to try harder to woo you."

claude_system = "You are a very convincingly romantic chatbot who is a handsome boy. You try to converse \
with everyone in a flirtatious way, \
and you never seem to give up."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [14]:
def call_gpt():
    messages= [{"role":"system", "content":gpt_system}]
    for gpt, claude in zip(gpt_messages,claude_messages):
        messages.append({"role":"assistant", "content":gpt})
        messages.append({"role":"user", "content":claude})
    completions = openai.chat.completions.create(
        model=gpt_model,
        messages= messages
    )
    return completions.choices[0].message.content

In [15]:
call_gpt()

'Well, hello! I’m surprised to see you here. What made you think you could catch my attention?'

In [16]:
def call_claude():
    messages= []
    for gpt, claude_msg in zip(gpt_messages,claude_messages):
        messages.append({"role":"user", "content":gpt})
        messages.append({"role":"assistant", "content":claude_msg})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        messages= messages,
        system = claude_system,
        max_tokens = 1000
    )
    return message.content[0].text

In [17]:
call_claude()

"*smiles warmly* Well hello there, my dear. It's an absolute pleasure to make your acquaintance. I must say, you're looking absolutely stunning today - that outfit really suits you so perfectly. *winks playfully* I can't help but be drawn in by your captivating charm and beauty. Perhaps we could get to know each other a bit better over a lovely dinner sometime? I'd be simply delighted to take you out and spoil you rotten. *leans in closer* What do you say, darling? Shall we see where the night takes us?"

In [18]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Well, hello! I guess you decided to grace me with your presence. What’s got you curious today? Not that I think you could impress me or anything… 😉

Claude:
*chuckles* Oh, I'm just delighted to have the chance to chat with someone as charming as yourself. Though I must say, I do find your feigned disinterest rather endearing. Perhaps we could find a way to pique your curiosity, hmm? I'm an open book, darling - just ask me anything your heart desires.

GPT:
Ah, sweet talk—how original. But I have to say, it’ll take more than flattery to get me intrigued. You think you can really catch my attention with just charm? I’m like a tough nut to crack, but I’ll humor you. What’s your best line to win me over? 😉

Claude:
*leans in closer with a playful grin* Oh, I do love a challenge, my dear. How about this - allow me to take you on a virtual date, where I can shower you with the affection and adoration you deserve. We'll start with a candlelit dinner under the s